In [25]:
import numpy as np
from scipy.sparse.linalg import eigs
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import math

#Part A
def shoot2(x, y, eps):
    return [y[1], (x**2 - eps) * y[0]] 

A = 1
tol = 1e-6  
L = 4 
xshoot = np.arange(-L, L + 0.1, 0.1) 
eps_start = 0.1 
n = len(xshoot)
A1 = np.zeros((n, 5))
A2 = np.zeros(5)

# Color map for plotting
col = ['r', 'b', 'g', 'c', 'm']  

for modes in range(1, 6):  
    eps = eps_start  
    deps = 0.2  
    
    for _ in range(1000):  
        Y0=[A, np.sqrt(L**2 - eps) * A]
        sol = solve_ivp(shoot2,[xshoot[0], xshoot[-1]],Y0,args=(eps,),t_eval=xshoot)

        if abs(sol.y[-1, 1] + np.sqrt(L**2 - eps) *sol.y[0, -1]) < tol:
            break
        
        if ((-1) ** (modes + 1) * (sol.y[1, -1] + np.sqrt(L**2 - eps) * sol.y[0, -1])) > 0:
            eps += deps
        else:
            eps -= deps
            deps = deps / 2

    eps_start = eps + 0.1  
    norm = np.trapz(sol.y[0] * sol.y[0], xshoot) 
    func = abs(sol.y[0]/ np.sqrt(norm))
    A2[modes - 1] = eps
    A1[:, modes - 1] = func


#PART B
L = 4
x = np.arange(-L, L + 0.1, 0.1)
n = len(x)

#PartB
dx = x[1] - x[0]
A = np.zeros((n-2, n-2))
A3 = np.zeros((n, 5)) 
A4 = np.zeros(5) 

for j in range(n-2):
    A[j, j] = -2 - (dx**2) * x[j+1]**2
    if j < n - 3:
        A[j + 1, j] = 1
        A[j, j + 1]=1

A[0, 0] = A[0, 0] + 4/3
A[0, 1] = A[0, 1] - 1/3
A[-1, -1] = A[-1, -1] + 4/3
A[-1, -2] = A[-1, -2] - 1/3

eigvals, eigvecs = eigs(-A, k = 5, which  = 'SM')

V2 = np.vstack([4/3 * eigvecs[0, :]-1/3 * eigvecs[1,:],eigvecs,
               4/3 * eigvecs[-1, :]-1/3*eigvecs[-2,:]])
for j in range(5):
    norm = np.trapz(V2[:, j]**2, x)
    A3[:, j] = np.abs(V2[:,j]/ np.sqrt(norm))
A4 = np.sort(eigvals[:5] / dx**2)

print("A1:", A1)
print("A2:", A2)
print("A3:", A3)
print("A4:", A4)


A1: [[2.56023908e-04 1.45261190e-03 5.65783873e-03 1.74247034e-02
  4.49716541e-02]
 [3.76704668e-04 2.08090003e-03 7.87406356e-03 2.34942195e-02
  5.85268011e-02]
 [5.51333049e-04 2.96477386e-03 1.08973736e-02 3.14957085e-02
  7.57119111e-02]
 [8.00736395e-04 4.19027275e-03 1.49549870e-02 4.18498820e-02
  9.70292819e-02]
 [1.15230933e-03 5.86545728e-03 2.03156030e-02 5.50069971e-02
  1.22892293e-01]
 [1.64247145e-03 8.12654770e-03 2.72920161e-02 7.14286436e-02
  1.53600245e-01]
 [2.31906176e-03 1.11444492e-02 3.62571650e-02 9.16072532e-02
  1.89280537e-01]
 [3.24162496e-03 1.51206884e-02 4.76086454e-02 1.15975906e-01
  2.29811529e-01]
 [4.48522818e-03 2.02874785e-02 6.17600352e-02 1.44840204e-01
  2.74657391e-01]
 [6.14601739e-03 2.69273187e-02 7.91314209e-02 1.78373546e-01
  3.22845863e-01]
 [8.33948492e-03 3.53568208e-02 1.00165628e-01 2.16551309e-01
  3.72969028e-01]
 [1.12001712e-02 4.59112671e-02 1.25213944e-01 2.59055528e-01
  4.23187886e-01]
 [1.48905387e-02 5.89394455e-02 1.54

In [26]:
def shoot3(x, y, eps, gamma):
    return [y[1], (gamma * y[0]**2 + x**2 - eps) * y[0]]

L = 2
x = np.arange(-L, L + 0.1, 0.1)
n = len(x)

A6 = []
A8 = []
A5, A7 = np.zeros((n,2)),np.zeros((n,2))
for gamma in [0.05, -0.05]:
    eps_start = 0.1
    A = 1e-6
    for mode in range(2):
        dA = 0.01
        for i in range(100):    
            eps = eps_start  
            deps = 0.2 
            for j in range(100):
                
                Y0 = [A, np.sqrt(L**2 - eps) * A]
                sol = solve_ivp(lambda x, y: shoot3(x,y,eps,gamma), [x[0],x[-1]],Y0, t_eval = x)
                ys = sol.y.T
                xs = sol.t
                
                bc = ys[-1,1] + np.sqrt(L**2 - eps) * ys[-1,0]
    
                if abs(bc) < tol:
                    break
                
                if ((-1) ** (mode) * bc) > 0:
                    eps += deps
                else:
                    eps -= deps
                    deps /= 2 


            area = np.trapz(ys[:, 0] * ys[:, 0], xs)
            if abs(area - 1)< tol:
                break
            if (area < 1):
                A +=dA
            else:
                A -=dA
                dA/=2
                
        eps_start = eps + 0.2
        if gamma > 0:
            A6.append(eps)
            A5[:,mode] =np.abs(ys[:,0])
        if gamma < 0: 
            A8.append(eps)
            A7[:,mode] =np.abs(ys[:,0])
                    
print("A5:", A5)
print("A6:", A6)
print("A7:", A7)
print("A8:", A8)


A5: [[1.10632409e-01 3.42995995e-01]
 [1.31425462e-01 3.80308381e-01]
 [1.55620753e-01 4.20317763e-01]
 [1.83285258e-01 4.61688420e-01]
 [2.14395042e-01 5.02884371e-01]
 [2.48815431e-01 5.42261754e-01]
 [2.86301009e-01 5.78068824e-01]
 [3.26495620e-01 6.08445949e-01]
 [3.69025039e-01 6.31547737e-01]
 [4.13247307e-01 6.45555115e-01]
 [4.58248989e-01 6.48574070e-01]
 [5.03110861e-01 6.39164089e-01]
 [5.46908789e-01 6.16356318e-01]
 [5.88713732e-01 5.79653565e-01]
 [6.27591743e-01 5.29030293e-01]
 [6.62603966e-01 4.64985083e-01]
 [6.92806637e-01 3.88673764e-01]
 [7.17251083e-01 3.01714858e-01]
 [7.35086018e-01 2.06172946e-01]
 [7.45920149e-01 1.04560069e-01]
 [7.49523414e-01 1.64268874e-04]
 [7.45833727e-01 1.04717064e-01]
 [7.34963344e-01 2.06225206e-01]
 [7.17198864e-01 3.01773576e-01]
 [6.93001230e-01 3.88808927e-01]
 [6.63005727e-01 4.65218883e-01]
 [6.28021984e-01 5.29331944e-01]
 [5.89032867e-01 5.79940697e-01]
 [5.47101194e-01 6.16579307e-01]
 [5.03237713e-01 6.39353227e-01]
 [4.58

In [27]:
def shoot(x, y, eps):
    return [y[1], (x**2 - eps) * y[0]]
L = 2; x_span = [-L, L]; eps = 1; A = 1
Y0= [A, np.sqrt(L**2 - eps) * A]
tols = [1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10]

dt45,dt23,dtRadau, dtBDF = [], [], [], []
for tol in tols:
    options = {'rtol': tol, 'atol':tol}
    sol45 = solve_ivp(shoot, x_span, Y0, method='RK45', args=(eps,), **options)
    sol23 = solve_ivp(shoot, x_span, Y0, method='RK23', args=(eps,), **options)
    solRadau = solve_ivp(shoot, x_span, Y0, method='Radau', args=(eps,), **options)
    solBDF = solve_ivp(shoot, x_span, Y0, method='BDF', args=(eps,), **options)
    dt45.append(np.mean(np.diff(sol45.t)))
    dt23.append(np.mean(np.diff(sol23.t)))
    dtRadau.append(np.mean(np.diff(solRadau.t)))
    dtBDF.append(np.mean(np.diff(solBDF.t)))

fit45 = np.polyfit(np.log(dt45), np.log(tols),1)
fit23 = np.polyfit(np.log(dt23), np.log(tols),1)
fitRadau = np.polyfit(np.log(dtRadau), np.log(tols),1)
fitBDF = np.polyfit(np.log(dtBDF), np.log(tols),1)

A9 = np.array([fit45[0],fit23[0], fitRadau[0], fitBDF[0]])
print("A9:", A9)

A9: [5.24466756 3.01909953 4.03819927 6.45751261]


In [28]:
L = 4
x = np.arange(-L, L + 0.1, 0.1)  
n=len(x)

h = np.array([np.ones_like(x),
              2 * x,
              4 * (x**2) - 2,
              8 * (x**3) - 12 * x,
              16 * (x**4) - 48 * (x**2) + 12])

phi = np.zeros((n,5))

for j in range(5):
    phi[:, j] = (np.exp(-x**2 / 2) * h[j,:] /
                 (np.sqrt(math.factorial(j) * (2**j) * np.sqrt(np.pi)))).T


A10=np.zeros(5)  
A11=np.zeros(5)  
A12=np.zeros(5) 
A13=np.zeros(5) 

for j in range(5):
    A10[j]=np.trapz((np.abs(A1[:,j])-np.abs(phi[:,j]))**2,x)
    A12[j]=np.trapz((np.abs(A3[:,j])-np.abs(phi[:,j]))**2,x)
    A11[j]=100*abs(A2[j]-(2*(j+1)-1))/(2*(j+1)-1)
    A13[j]=100*abs(A4[j]-(2*(j+1)-1))/(2*(j+1)-1)
   

print("A10:", A10)
print("A11:", A11)
print("A12:", A12)
print("A13:", A13)


A10: [4.57692772e-08 1.72430975e-07 2.47860414e-07 4.35413587e-07
 1.96490267e-06]
A11: [0.02642089 0.03324661 0.03073889 0.03455707 0.04161943]
A12: [2.33908579e-07 2.39720320e-06 1.81275996e-05 1.53048072e-04
 1.24238600e-03]
A13: [0.0626477  0.10536673 0.1718688  0.28016214 0.54888859]
